In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime

import sys

sys.path.append('../')

from implementations import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# **Load and clean the training data**

We load the training data.

In [3]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("../data")

print(f"The data has {x_train.shape[0]} samples and {x_train.shape[1]} features !")

The data has 328135 samples and 321 features !


We then clean the data by : 
- removing the nan values by the mean of the rest of the feature
- removing the features where the variance is zero since they are constants for all samples
- remove the 8 first features as the appear weird in the task of predicting a heart attack

In [6]:
cont_data = [13, 15, 16, 17, 24 ,25 ,26 ,27 ,28 ,29 ,33 ,37,49 ,59 ,60 ,62 ,63 ,75 ,77 ,78 ,
             79 ,80 ,81 ,82 ,83 ,84 ,85 ,86 ,89 ,90 ,92 ,93 ,94, 98 ,110 ,111 ,112 ,113 ,114 ,143
            ,147 ,148 ,149 ,150 ,168 ,195 ,197 , 206 ,207 ,208 ,209 ,210 ,211 ,212 ,213 ,219, 220, 221
            ,222, 226, 228, 229, 248 ,250 ,251 ,252 ,253 ,262 ,264 ,266,267,268,269,270,271,276,277,285,286,287,288,291,292, 293,294, 295, 296, 297, 299, 300, 301, 302, 303, 304]

# Generate a list of all indices from 1 to 320
all_indices = list(range(0, 321))

# Use a list comprehension to filter out indices not in your array
cat_data = [idx for idx in all_indices if idx not in cont_data]

In [7]:
xt_feat = x_train.copy()
xt_feat = preprocessing(xt_feat)
xt_feat = gen_binary(xt_feat, cat_data, cont_data)
print(f"The data has now {xt_feat.shape[1]} features !")

The data has now 321 features !


In [8]:
xt_z = xt_feat.copy()
for i in np.array([0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 19, 20, 21, 22, 23, 88, 91, 101, 105]):
    xt_z[:, i] = 0

In [9]:
xt_feat_sep = cat_sep(xt_z, cat_data)
print(f"The data has now {xt_feat_sep.shape[1]} features !")

Feature : 0
Feature : 1
Feature : 2
Feature : 3
Feature : 4
Feature : 5
Feature : 6
Feature : 7
Feature : 8
Feature : 9
Feature : 10
Feature : 11
Feature : 12
Feature : 14
Feature : 18
Feature : 19
Feature : 20
Feature : 21
Feature : 22
Feature : 23
Feature : 30
Feature : 31
Feature : 32
Feature : 34
Feature : 35
Feature : 36
Feature : 38
Feature : 39
Feature : 40
Feature : 41
Feature : 42
Feature : 43
Feature : 44
Feature : 45
Feature : 46
Feature : 47
Feature : 48
Feature : 50
Feature : 51
Feature : 52
Feature : 53
Feature : 54
Feature : 55
Feature : 56
Feature : 57
Feature : 58
Feature : 61
Feature : 64
Feature : 65
Feature : 66
Feature : 67
Feature : 68
Feature : 69
Feature : 70
Feature : 71
Feature : 72
Feature : 73
Feature : 74
Feature : 76
Feature : 87
Feature : 88
Feature : 91
Feature : 95
Feature : 96
Feature : 97
Feature : 99
Feature : 100
Feature : 101
Feature : 102
Feature : 103
Feature : 104
Feature : 105
Feature : 106
Feature : 107
Feature : 108
Feature : 109
Feature : 11

In [12]:
separated_categories = np.delete(xt_feat_sep, cat_data, axis=1)
x_train_pca = separated_categories
print(f"The data has now {separated_categories.shape[1]} features !")

The data has now 814 features !


# PCA algorithm implementation

In [24]:
pca_indices, idx = pca(separated_categories)
print(f"We can keep the {idx} first most influent features given by pca_indices")

We can keep the 138 first most influent features given by pca_indices


In [25]:
x_train_pca = separated_categories[:, pca_indices]
x_train_pca = separated_categories[:, :idx]

print(f"The data has now {x_train_pca.shape[1]} features")

The data has now 138 features


# Logistic regression using stochastic gradient descent (SGD)

We train our model using logistic regression using SGD with mean-square error.

First, we separate our data in a training set(70%) and testing set(30%).

In [26]:
tx_tr, tx_te, y_tr, y_te = cross(x_train_pca, y_train, 0.48)

print(f"tx_tr shape : {tx_tr.shape} ; tx_te shape : {tx_te.shape}")
print(f"       y_tr : {y_tr.shape}     ;        y_te : {y_te.shape}")

tx_tr shape : (157504, 138) ; tx_te shape : (170631, 138)
       y_tr : (157504,)     ;        y_te : (170631,)


Now we build our models for linear regression using SGD

In [27]:
y, tx = build_model_data(tx_tr, y_tr)
y_test, tx_test = build_model_data(tx_te, y_te)
print(f"The data has now {tx.shape[1]} features !")

The data has now 139 features !


## Training 

Here we train our model using SGD with MSE

In [ ]:
initial_w = np.zeros(tx.shape[1])
degree = range(1,4)
gammas = np.logspace(-5, 0, 15)
max_iters = 2000

start_time = datetime.datetime.now()
best_degree, best_gamma, best_loss = logistic_regression_demo(tx, y, gammas, degree, max_iters)
end_time = datetime.datetime.now()

print(
    "The best rmse of %.3f is obtained for a degree of %.f and a lambda of %.5f."
    % (best_loss, best_degree, best_gamma)
)

print(f"Execution time {(end_time - start_time).total_seconds()} seconds")

In [38]:
# best_gamma = 6e-06
# max_iters = 1201
best_gamma = 3e-05
initial_w = np.zeros(tx.shape[1])
max_iters = 801

w, loss = logistic_regression(y, tx, initial_w, max_iters, best_gamma)

Current iteration=0, loss=0.6931471805599453
Current iteration=100, loss=0.599260986701567


### Computation of metrics

We first compute some metrics on the training data (60% of the total data)

In [39]:
best_thresh = best_threshold(y, tx, w)

In [40]:
pred_data = np.dot(tx, w)

pred_data[pred_data > best_thresh] = 1
pred_data[pred_data <= best_thresh] = -1

correctly_classified_data = np.sum(pred_data == y)

tp = np.sum((pred_data == 1) & (y == 1))
fp = np.sum((pred_data == 1) & (y == -1))

tn = np.sum((pred_data == -1) & (y == -1))
fn = np.sum((pred_data == -1) & (y == 1))

accuracy_data = (tp + tn)/(tp + fp + tn + fn)

print(f"Accuracy: {accuracy_data*100}%")
print(f"Precision: {tp/(tp + fp)*100}%")
print(f"Recall : {tp/(tp + fn)*100}%")
print(f"F1-score : {tp/(tp + 0.5*(fn + fp))*100}%")

Accuracy: 63.47457842340511%
Precision: 14.545454545454545%
Recall : 64.93845039826212%
F1-score : 23.7673093487047%


Now we compute some metrics for our test data (40% of the total data)

In [41]:
pred_test = np.dot(tx_test, w)

pred_test[pred_test > best_thresh] = 1
pred_test[pred_test <= best_thresh] = -1

correctly_classified_test = np.sum(pred_test == y_test)

tp = np.sum((pred_test == 1) & (y_test == 1))
fp = np.sum((pred_test == 1) & (y_test == -1))

tn = np.sum((pred_test == -1) & (y_test == -1))
fn = np.sum((pred_test == -1) & (y_test == 1))

accuracy_test = (tp + tn)/(tp + fp + tn + fn)

print(f"Accuracy: {accuracy_test*100}%")
print(f"Precision: {tp/(tp + fp)*100}%")
print(f"Recall : {tp/(tp + fn)*100}%")
print(f"F1-score : {tp/(tp + 0.5*(fn + fp))*100}%")

Accuracy: 63.58281906570319%
Precision: 14.741645900261208%
Recall : 64.75436861193538%
F1-score : 24.015945413859306%


# Prediction on test data

In [ ]:
start_time = datetime.datetime.now()

tx_test = x_test
tx_test = preprocessing(x_test)
tx_test = gen_clean(tx_test, [], np.arange(321))

tx_test = tx_test[:, pca_indices]
tx_test = tx_test[:, :idx]
tx_test = np.c_[np.ones(tx_test.shape[0]), tx_test]

end_time = datetime.datetime.now()
print(f"Execution time {(end_time - start_time).total_seconds()} seconds")
print(f"The data has {tx_test.shape[0]} samples and {tx_test.shape[1]} features !")

In [ ]:
pred_te = np.dot(tx_test, w)

pred_te[pred_te > best_thresh] = 1
pred_te[pred_te < best_thresh] = -1

indices_one = np.where(pred_te == 1)

In [94]:
create_csv_submission(test_ids, pred_te, "../data/log_reg.csv")